In [17]:
import pandas as pd     
import numpy as np     
import re     
import jieba     
import matplotlib.pyplot as plt     
import pyLDAvis     
import pyLDAvis.gensim_models as gensimvis

In [18]:
df = pd.read_csv('C:/Users/20693/Desktop/智慧养老政策.csv', encoding='gbk',index_col = 0)
df

,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接
序号,,,,,,,,,,
地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...
地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...
地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...
地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...
地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...
...,...,...,...,...,...,...,...,...,...,...
地方工作文件_15,郑州市人民政府关于印发郑州市数字政府建设行动方案的通知,现行有效,地方工作文件,郑州市人民政府,郑政〔2023〕25号,2023.12.25,2023.12.25,NaN,NaN,https://www.pkulaw.com/lar/b7908e42b1682d8bafd...
部门规章_1,产业结构调整指导目录(2024年本),尚未施行,部门规章,国家发展和改革委员会(含原国家发展计划委员会、原国家计划委员会),中华人民共和国国家发展和改革委员会令第7号,2023.12.27,2024.02.01,NaN,NaN,https://www.pkulaw.com/chl/d8513a2dd8e4e297bdf...
地方工作文件_14,山西省人民政府办公厅关于印发山西省数字政府建设规划(2023―2025年)的通知,现行有效,地方工作文件,山西省人民政府,晋政办发〔2023〕82号,2023.12.28,2023.12.28,NaN,NaN,https://www.pkulaw.com/lar/cf041968e90d5c7d666...


####语料预处理，提出信息中的“噪音”

In [19]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题'][0]
remove_nums(test)[:100]

C:\Users\20693\AppData\Local\Temp/ipykernel_18652/3651434706.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test = df['标题'][0]


'黑龙江省人民政府办公厅关于推进养老服务发展的实施意见'

In [20]:
def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题'][0]
clean_text(test)

C:\Users\20693\AppData\Local\Temp/ipykernel_18652/2424506813.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test = df['标题'][0]


'黑龙江省 人民政府 办公厅 关于 推进 养老 服务 发展 的 实施 意见'

In [21]:
df['标题'] = df['标题'].astype(str)
df['标题分词'] = df['标题'].apply(remove_nums)
df['标题分词'] = df['标题分词'].apply(clean_text)
df['标题分词'] = df['标题分词'].apply(lambda x: x.split())
df

,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接,标题分词
序号,,,,,,,,,,,
地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...,"[黑龙江省, 人民政府, 办公厅, 关于, 推进, 养老, 服务, 发展, 的, 实施, 意见]"
地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...,"[广西壮族自治区, 大, 数据, 发展局, 关于, 全面, 推进, 数字化, 转型, 发展,..."
地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...,"[中共, 无锡市委, 无锡市, 人民政府, 印发, 关于, 加快, 推进, 数字, 经济, ..."
地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...,"[贵阳市, 民政局, 关于, 推进, 贵阳, 贵安, 养老, 服务, 高质量, 发展, 的,..."
地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...,"[廊坊市, 人民政府, 办公室, 关于, 加快, 推进, 养老, 服务体系, 建设, 的, ..."
...,...,...,...,...,...,...,...,...,...,...,...
地方工作文件_15,郑州市人民政府关于印发郑州市数字政府建设行动方案的通知,现行有效,地方工作文件,郑州市人民政府,郑政〔2023〕25号,2023.12.25,2023.12.25,NaN,NaN,https://www.pkulaw.com/lar/b7908e42b1682d8bafd...,"[郑州市, 人民政府, 关于, 印发, 郑州市, 数字, 政府, 建设, 行动, 方案, 的..."
部门规章_1,产业结构调整指导目录(2024年本),尚未施行,部门规章,国家发展和改革委员会(含原国家发展计划委员会、原国家计划委员会),中华人民共和国国家发展和改革委员会令第7号,2023.12.27,2024.02.01,NaN,NaN,https://www.pkulaw.com/chl/d8513a2dd8e4e297bdf...,"[产业, 结构调整, 指导, 目录, 年本]"
地方工作文件_14,山西省人民政府办公厅关于印发山西省数字政府建设规划(2023―2025年)的通知,现行有效,地方工作文件,山西省人民政府,晋政办发〔2023〕82号,2023.12.28,2023.12.28,NaN,NaN,https://www.pkulaw.com/lar/cf041968e90d5c7d666...,"[山西省人民政府, 办公厅, 关于, 印发, 山西省, 数字, 政府, 建设, 规划, 年,..."


In [31]:
from gensim import corpora, models

ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\Users\20693\AppData\Roaming\Python\Python39\site-packages\scipy\linalg\__init__.py)

###救命啊不知道咋回事我的电脑死活装不上gensim包了呜呜呜，重装、更新和换版本都不行